In [ ]:
import pandas as pd
import numpy as np
import folium 
import geopandas as gpd
import folium.plugins as plugins

from datetime import datetime


from difflib import get_close_matches

import matplotlib.pyplot as plt
import matplotlib.pyplot

%matplotlib inline

# import chart_studio.plotly as py
# import plotly.graph_objects as go

import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

from scipy.stats.mstats import winsorize

In [ ]:
#load tech_1
tech1 = pd.read_csv(
    r'/mnt/c/Users/handy019/OneDrive - University of South Australia/Python/3 webscrapping/3 summary/tech_1.csv',
    encoding='iso-8859-1')

#load location tech1 (for non-similar names only (around 2000))
loc_tech1 = pd.read_csv(
    r'/mnt/c/Users/handy019/OneDrive - University of South Australia/Python/3 webscrapping/3 summary/loc_for_tech_1.csv',
    encoding='iso-8859-1')

print(len(loc_tech1))

#prepare tech_2

df1 = pd.read_csv(
    r'/mnt/c/Users/handy019/OneDrive - University of South Australia/Python/3 webscrapping/4 do it again/spacy name entities/name_entity_proquest_tech(2).csv',
    encoding='iso-8859-1')
df1_1 = pd.DataFrame()

#prep the data for organization name
for i in range(len(df1)):

    x = df1['organization'].loc[i]
    list_1 = []

    if type(x) != float:
        list_1 = list_1 + [y.strip("''") for y in x.split(', ')]
        temp = pd.DataFrame({'org': list_1})
        temp['tech'] = df1.tech.loc[i]
        df1_1 = pd.concat([df1_1, temp], ignore_index=True)
    else:
        pass

test1 = pd.read_csv(
    r'/mnt/c/Users/handy019/OneDrive - University of South Australia/Python/3 webscrapping/2 newspaper/technews(2)_1000.csv',
    encoding='iso-8859-1')
test2 = pd.read_csv(
    r'/mnt/c/Users/handy019/OneDrive - University of South Australia/Python/3 webscrapping/2 newspaper/technews(2)_2000.csv',
    encoding='iso-8859-1')
test3 = pd.read_csv(
    r'/mnt/c/Users/handy019/OneDrive - University of South Australia/Python/3 webscrapping/2 newspaper/technews(2)_3000.csv',
    encoding='iso-8859-1')

pq_link = pd.concat([test1,test2,test3], ignore_index=True)

#merge date time with organization name
print(len(df1))
df1 = pd.merge(df1, pq_link[['tech','link','date_']], how='left', on=['tech','link'])
print(len(df1))

df1 = df1.reset_index()

# load location of organization (for non-similar name only)
loc1 = pd.read_csv(
    r'/mnt/c/Users/handy019/OneDrive - University of South Australia/Python/3 webscrapping/4 do it again/spacy name entities/geo_location(2)_1.csv',
    encoding='iso-8859-1')

print(len(loc1))

# more location for organization (from similar name)
loc_similar1 = pd.read_csv(
    r'/mnt/c/Users/handy019/OneDrive - University of South Australia/Python/3 webscrapping/4 do it again/spacy name entities/geo_location_similar.csv',
    encoding='iso-8859-1')

print(len(loc_similar1))

In [ ]:
# print(len(df1))
# print(len(tech1))
# df1 = pd.concat([df1,tech1],ignore_index=True)
# print(len(df1))
# df1 = df1.drop_duplicates(ignore_index=True)
# print(len(df1))
# df1.head()

In [ ]:
print(len(loc1))
print(len(loc_tech1))
print(len(loc_similar1))

loc = pd.concat([loc1,loc_tech1,loc_similar1],ignore_index=True)

print(len(loc))
loc=loc.drop_duplicates(ignore_index=True)

print(len(loc))

In [ ]:
loc.head()

In [ ]:
df1.tech.unique()

In [ ]:
Aerospace_Space = [
    'satellite technology', 'aerospace engineering', 'space exploration'
]
Advanced_manufacturing = [
    'smart technology', 'smart home technology', 'automation systems',
    'smart technologies', 'photovoltaic panels', 'robotics'
]
Agtech = ['agtech']
Digital_infrastructure = [
    'blockchain', 'crowd funding', 'cloud-based software', 'data storage',
    'blockchain innovation'
]
Health_medical = ['digital health', 'neuroplasticity']
Machine_learning = [
    'data analytics',
    'autonomous vehicle technology',
    'automation systems',
    'blockchain innovation',
    'aerial-mapping',
    'e-invoicing',
    'robotics',
    'point of sale',
    'artificial intelligence',
    'smart technology',
    'smart home technology',
    'smart technologies',
]
Renewable_energy = ['energy storage technology', 'renewable energy']
VFX_VR_AR = ['VR technology', 'virtual reality', 'augmented reality']
start_up = ['start-up']

In [ ]:
#GroupedLayerControl
from jinja2 import Template
from branca.element import Figure
from folium.map import Layer, LayerControl, FeatureGroup

class GroupedLayerControl(Layer):
	_template = Template(u"""
			{% macro header(this, kwargs) %}
			<link rel="stylesheet" href="https://raw.githubusercontent.com/ismyrnow/leaflet-groupedlayercontrol/gh-pages/src/leaflet.groupedlayercontrol.css" />
			<script src="leaflet.groupedlayercontrol.js"></script> 
			{% endmacro %}
			{% macro script(this, kwargs) %}
			var baseMaps = { {%- for groupkey, group in this._basemaps.items() %}
							{{ groupkey|tojson }} : {		
										 {%- for key, val in group.items() %}
											{{ key|tojson }} : {{ val.get_name() }},
										{%- endfor %}
								}	,
							{%- endfor %}
			};
			var overlays = {
					 {%- for groupkey, group in this._overlays.items() %}
							
								 {{ groupkey|tojson }} : {		
										 {%- for key, val in group.items() %}
											{{ key|tojson }} : {{ val.get_name() }},
										{%- endfor %}
									}  ,
							{%- endfor %} };
			var options = {
				exclusiveGroups : [ {%- for i in this._radio %} "{{i}}", {%- endfor %} ],
				container_width 	: "300px",
				group_maxHeight     : "80px",
				exclusive       	: false,
				collapsed : true, 
				position: 'topright',
				show: false
			};
			
			L.control.groupedLayers(baseMaps, overlays, options).addTo({{ this._parent.get_name() }})
			
			document.getElementById('idname').click();
        {% endmacro %}
        """)
		
	def __init__(self, bmaps, olays, radio, name=None, overlay=True, control=True, show=True, options=None, **kwargs):
		super(GroupedLayerControl, self).__init__(name=name, overlay=overlay, control=control, show=show)
		self._basemaps = bmaps
		self._overlays = olays
		self._radio = radio
		self._name = 'GroupedLayerControl'

In [ ]:
def create_groupdate(df, key_list):

    #prepare
    groupdate = df1[(~df1['organization'].isna())
                   & (df1['tech'].isin(Aerospace_Space))].groupby(
                       'date_')['organization'].apply(','.join).reset_index()
    groupdate = groupdate.drop(axis=1, index=len(groupdate) - 1)

    groupdate['year'] = groupdate['date_'].apply(
        lambda x: x.split(',')[1].strip(" "))

    groupdate2 = groupdate.groupby('year')['organization'].apply(
        ','.join).reset_index()

    return groupdate2

In [ ]:
def prepare_map(groupdate2, sa_only, a):
    #get the frame for the map
    new = pd.DataFrame()

    for i in range(len(sa_only)):
        try:
            temp = groupdate2['organization'].apply(lambda x: x.count(sa_only['big_list_name'].loc[i]))

            temp1 = groupdate2[['year']].copy()
            temp1['weight'] = temp
            temp1['lat'] = sa_only['geometry.location.lat'].loc[i]
            temp1['lng'] = sa_only['geometry.location.lng'].loc[i]
            temp1['saindex'] = sa_only['index'].iloc[i]

            new = pd.concat([new,temp1],ignore_index=True)
        except:
            print(i)

    #get the data for time-lapse heat map
    list_ = []
    year_ = []

    new['weight_2'] = winsorize((new['weight']), limits=[0, 0.005]).tolist()

    new = new[new.weight_2!=0].reset_index(drop=True).copy()
    new['weight_2'] = new['weight_2'] /new['weight_2'].abs().max()
    new = new.sort_values(['year']).reset_index(drop=True)
    
    return new

In [ ]:
sa_only = loc.reset_index(drop=True)
sa_only = sa_only[['geometry.location.lat','geometry.location.lng','big_list_name']]
sa_only = sa_only.reset_index()
sa_only = sa_only[~sa_only['big_list_name'].isna()].reset_index(drop=True)

a = 0.005

frame1 = prepare_map(create_groupdate(df1, Aerospace_Space), sa_only, a)
frame1['tech'] = 'Aerospace Space and Space'

In [ ]:
frame2 = prepare_map(create_groupdate(df1, Advanced_manufacturing), sa_only, a)
frame2['tech'] = 'Advanced manufacturing'

In [ ]:
frame3 = prepare_map(create_groupdate(df1, Agtech), sa_only, a)
frame3['tech'] = 'Agtech'

In [ ]:
frame4 = prepare_map(create_groupdate(df1, Digital_infrastructure), sa_only, a)
frame4['tech'] = 'Digital infrastructure'

In [ ]:
frame5 = prepare_map(create_groupdate(df1, Health_medical), sa_only, a)
frame5['tech'] = 'Health and medical technology'

In [ ]:
frame6 = prepare_map(create_groupdate(df1, Machine_learning), sa_only, a)
frame6['tech'] = 'Machine learning and big data'

In [ ]:
frame7 = prepare_map(create_groupdate(df1, Renewable_energy), sa_only, a)
frame7['tech'] = 'Renewable energy'

In [ ]:
frame8 = prepare_map(create_groupdate(df1, VFX_VR_AR), sa_only, a)
frame8['tech'] = 'VFX, VR, and AR'

In [ ]:
frame9 = prepare_map(create_groupdate(df1, start_up), sa_only, a)
frame9['tech'] = 'start up'

In [ ]:
final_frame = pd.concat([frame1,frame2,frame3,frame4,frame5,frame6,frame7,frame8,frame9], ignore_index=True)

In [ ]:
#final time-lapse heatmap
# m = folium.Map([-25.714884, 135.219674], tiles=None, zoom_start=6)

# folium.TileLayer('CartoDB positron', name="Light Map", control=False).add_to(m)

# # hm = plugins.HeatMapWithTime(list_, index= year_, radius=5, blur=0.2, min_opacity=0.2)
# hm = plugins.HeatMapWithTime(list_, index= year_, radius=5, blur=0.2, min_opacity=0.5 , use_local_extrema=False)

# fg1 = folium.map.FeatureGroup(name='Group 1', show = True)

# folium.plugins.HeatMap(final_frame[(final_frame['tech']=='Aerospace Space and Space') & (final_frame['year']=='2019')][['lat','lng','weight_2']].to_numpy(), radius=5, blur=0.2, min_opacity=0.5 , use_local_extrema=False).add_to(fg1)

# fg1.add_to(m)

# fg2 = folium.map.FeatureGroup(name='Group 2', show = True)

# folium.plugins.HeatMap(final_frame[(final_frame['tech']=='Aerospace Space and Space') & (final_frame['year']=='2020')][['lat','lng','weight_2']].to_numpy(), radius=5, blur=0.2, min_opacity=0.5 , use_local_extrema=False)

# fg2.add_to(m)

# GroupedLayerControl({}, {'Technology' : {'Aerospace Space and Space (2019)' : fg1 , 'Aerospace Space and Space (2020)' : fg2}
#                         }, ['Category 1']).add_to(m)


In [ ]:
# import pandas as pd
# import numpy as np
# import plotly.express as px

# fig = px.density_mapbox(final_frame[(final_frame['tech']=='Aerospace Space and Space') & (final_frame['year']=='2019')], lat='lat', lon='lng', z='weight_2',
#                         mapbox_style="stamen-terrain", opacity=0.5)
 
# fig

In [ ]:
# import os
# import folium

# import numpy as np
# import random

# # Generate base data
# data = (np.random.normal(size=(100, 2)) * np.array([[1, 1]]) +
#                 np.array([[48, 5]]))
# # Generate the data to segment by (levels of another pandas column in practical usage)
# categories = ['category{}'.format(i+1) for i in range(5)]
# category_column = [random.choice(categories) for i in range(len(data))]


In [ ]:
# from folium.plugins import TagFilterButton

# # Create map and add the data with additional parameter tags as the segmentation
# m = folium.Map([48., 5.], tiles='stamentoner', zoom_start=7)
# for i, latlng in enumerate(data):
#     category = category_column[i]
#     folium.Marker(
#         tuple(latlng),
#         tags=[category]
#     ).add_to(m)
    
# TagFilterButton(categories).add_to(m)

# m


In [ ]:
# import os
# import random

# import folium
# from folium import plugins
# import pandas as pd
# import numpy as np

# # Generate base data
# data_df = pd.DataFrame(np.random.normal(size=(100, 2)) * np.array([[1, 1]]) + np.array([[48, 5]]),
#                        columns=['longitude', 'latitude'])

# # Generate the data to segment by (levels of another pandas column in practical usage)
# categories = ['category{}'.format(i+1) for i in range(5)]
# category_column = [random.choice(categories) for i in range(len(data_df))]
# data_df['category'] = category_column

# # Split data into 2 groups
# data_df_gp1 = data_df.loc[:len(data_df)/2, :]
# data_df_gp2 = data_df.loc[len(data_df)/2:, :]

# # Base map
# m = folium.Map([48., 5.], tiles=None, zoom_start=7)

# # Create 2 featuregroup
# fg1 = folium.FeatureGroup(name="Group1", overlay=False).add_to(m)
# fg2 = folium.FeatureGroup(name="Group2", overlay=False).add_to(m)

# # Plot on feature group 1
# folium.TileLayer().add_to(fg1)
# for y in ['2019','2020']:
#     category = row['category']
#     folium.Marker(
#         tuple([row['longitude'], row['latitude']]),
#         tags=[category]
#     ).add_to(fg1)
    
# # Plot on feature group 2
# folium.TileLayer().add_to(fg2)
# for name, row in data_df_gp2.iterrows():
#     category = row['category']
#     folium.Marker(
#         tuple([row['longitude'], row['latitude']]),
#         tags=[category]
#     ).add_to(fg2)

# # Add LayerControl
# folium.LayerControl(collapsed=False).add_to(m)

# # Add TagFilterButton
# plugins.TagFilterButton(categories, control=False).add_to(m)

# # show map
# m

In [ ]:
import os
import random

import folium
from folium import plugins
import pandas as pd
import numpy as np

# Generate base data
data_df = final_frame[final_frame['tech'] == 'Aerospace Space and Space']

# Split data into 2 groups
data_df_gp1 = data_df[data_df['tech'] == 'Aerospace Space and Space']

# Base map
m = folium.Map([-25.714884, 135.219674], tiles=None, zoom_start=6)

folium.TileLayer('CartoDB positron', name="Light Map", control=False).add_to(m)

# Create 2 featuregroup
for y in ['2019', '2020', '2021', '2022']:
    folium.plugins.HeatMap(
        data_df_gp1[data_df_gp1['year'] == y][['lat', 'lng',
                                               'weight_2']].to_numpy(),
        radius=10,
        blur=5).add_to(
            folium.FeatureGroup(name='{}'.format(y)).add_to(m))

# # Plot on feature group 2
# for y in ['2019', '2020', '2021', '2022']:
#     folium.plugins.HeatMap(data_df_gp2[data_df_gp2['year'] == y][[
#         'lat', 'lng', 'weight_2'
#     ]].to_numpy()).add_to(folium.FeatureGroup(name='{}'.format(y)).add_to(m))

# Add LayerControl
folium.LayerControl(collapsed=False).add_to(m)

# show map
m

In [1]:
final_frame

NameError: name 'final_frame' is not defined